In [ ]:
#importing required libraries
import cv2
import tensorflow as tf
import numpy as np

#Loading model Eyemodell
new_model =tf.keras.models.load_model('Eyemodell.h5')

#assigning classifier
path = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

#assigning vediocapture to cap                                
cap = cv2.VideoCapture(1)
  
#createing condition to open camera    
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")
        
#creating while loop for detecting the face and eyes        
while True:
    ret,frame = cap.read()
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_eye.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes=eye_cascade.detectMultiScale(gray,1.1,4)
    for x,y,w,h in eyes:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        eyess = eye_cascade.detectMultiScale(roi_gray)
        if len(eyess) == 0:
            print ("eyes are not detected")
        else:
            for(ex,ey,ew,eh) in eyess:
                eyes_roi = roi_color[ey: ey+eh, ex:ex + ew]
                #resizing image 
                final_image = cv2.resize(eyes_roi,(224, 224))
                final_image = np.expand_dims(final_image,axis =0)
                #Normalizing image 
                final_image=final_image/255.0            
                #creating prediction system & assigning with status          
                Predictions = new_model.predict(final_image)
                if (Predictions>0):
                    status = "Open Eyes"
                else:
                    status = "Closed Eyes"
                                    
                                    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    print(faceCascade.empty())
    faces = faceCascade.detectMultiScale(gray,1.1,4)

    # Draw a rectangle around the faces
    for(x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    font = cv2.FONT_HERSHEY_SIMPLEX

    #Adding status font                                
    cv2.putText( frame, status, (58, 50), font, 3, (0, 0, 255), 2, cv2.LINE_4)
    cv2.imshow( 'Drowsiness Detection Tutorial',frame)

    if cv2.waitKey(2) & 0XFF == ord('q'):
        break
                                    
cap.release()
cv2.destroyAllwindows()  